In [1]:
import pandas as pd
import geopandas as gpd
import time
import os
import warnings
from pandas import read_csv

warnings.filterwarnings("ignore")
os.chdir('/Users/xiaodanxu/Library/CloudStorage/GoogleDrive-arielinseu@gmail.com/My Drive/GEMS/BILD-AQ/data')
# selected_state = 'MS'

In [2]:
microtype_lookup = read_csv('ccst_geoid_key_tranps_geo_with_imputation.csv', sep = ',')
# state_list = microtype_lookup['st_code'].unique()
# state_list = ['AZ', 'NV']

In [3]:
census_tract_shapefile = 'Network/combined/census_tracts_2017.geojson'
polygon_gdf = gpd.read_file(census_tract_shapefile)
polygon_gdf = polygon_gdf.to_crs('EPSG:4326')
# polygon_gdf.plot()
# from pygris.utils import shift_geometry

# polygon_gdf_rescaled = shift_geometry(polygon_gdf)
# polygon_gdf_rescaled.plot()

In [4]:
# # perform spatial intersection
# for selected_state in state_list:
#     if selected_state == 'OH':
#         continue
#     print('processing state ' + selected_state)
output_dir = 'route/processed/final'
route_list = [file for file in os.listdir(output_dir) if file.endswith('.geojson')]
#     print(route_list[0:5])
start_time = time.time()
for route in route_list:
    if route.startswith('._'):
        continue
    file_name = route.split('.geojson')[0]
    isExist = os.path.exists(output_dir + '/' + file_name + '.csv')
    if isExist:
        continue
    print('processing route ' + route)
    line_gdf = gpd.read_file(output_dir + '/' + route)
#     sample_line_gdf = line_gdf.head(100)
    # print(start_time)
    line_by_polygon = gpd.overlay(line_gdf, polygon_gdf, how='intersection')

    # compute segment length in meters
    line_by_polygon = line_by_polygon.to_crs("EPSG:3310") 
    # in order to get length in meter, the shapefile need to re-projected to a coordinate system in meters (not required in R)
    line_by_polygon.loc[:, 'Length'] = line_by_polygon.loc[:, 'geometry'].length

    line_by_polygon_df = pd.DataFrame(line_by_polygon.drop(columns='geometry'))
    line_by_polygon_df = line_by_polygon_df[['source',	'destination',
                                              'distance', 'GEOID', 'Length']]
#     line_by_polygon_df.columns = []
#     print(len(line_by_polygon))
    line_by_polygon_df.to_csv(output_dir + '/' + file_name + '.csv', index = False)

#     break
end_time = time.time()
total_time = end_time - start_time
print('spatial intersection in Python takes ' + str(total_time) + ' sec')

processing route imputed_NV_route_1.geojson
processing route imputed_CO_route_1.geojson
processing route imputed_AZ_route_1.geojson
spatial intersection in Python takes 477.15525794029236 sec


In [6]:
line_by_polygon_df.head(5)

,source,destination,distance,GEOID,Length
0,16079960400,16079960200,100833.2,16079960200,60860.539213
1,16079960300,16079960200,102781.4,16079960200,65016.490690
2,16079960200,16055000900,159196.7,16079960200,73787.743107
3,16079960400,16055001900,69820.4,16079960200,8771.252416
4,16079960200,16079960400,100849.8,16079960200,60860.539213


In [6]:
line_gdf.head(5)

,distance,duration,source,destination,geometry
0,135259.9,7577.1,06015000201,41033360600,"LINESTRING (-124.15712 41.84009, -124.15722 41..."
1,385536.9,23947.3,06093000100,32031002204,"LINESTRING (-121.64985 41.78186, -121.65011 41..."
2,256453.3,11497.4,06065046200,04015953900,"LINESTRING (-114.59659 33.59947, -114.59664 33..."
3,272950.9,11109.9,06065045900,04013217201,"LINESTRING (-114.67313 33.53007, -114.66508 33..."
4,286776.8,11829.3,06065045900,04013216816,"LINESTRING (-114.67313 33.53007, -114.66508 33..."
